In [1]:
def read_knapsack_data(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # İlk satır: öğe sayısı ve kapasite
    n, capacity = map(int, lines[0].split())

    # Sonraki satırlar: değer ve ağırlıklar
    values, weights = [], []
    for line in lines[1:]:
        v, w = map(int, line.strip().split())
        values.append(v)
        weights.append(w)

    return n, capacity, values, weights

# 📥 Veri dosyasını oku
filepath = "ks_300_0.txt"
n, capacity, values, weights = read_knapsack_data(filepath)

# 🧮 Knapsack çözümü (Dynamic Programming)
dp = [0] * (capacity + 1)
item_taken = [[False] * (capacity + 1) for _ in range(n)]

for i in range(n):
    for w in range(capacity, weights[i] - 1, -1):
        if dp[w - weights[i]] + values[i] > dp[w]:
            dp[w] = dp[w - weights[i]] + values[i]
            item_taken[i][w] = True

# 🔍 Geriye izleme: Hangi item'lar alındı?
w = capacity
selected_items = []
for i in range(n - 1, -1, -1):
    if item_taken[i][w]:
        selected_items.append(i)
        w -= weights[i]

selected_items.reverse()

# ✅ Sonuçlar
print(f"✅ Optimal Değer: {dp[capacity]}")
print(f"📋 Seçilen Öğeler (0-index): {selected_items}")
print(f"💰 Seçilen Değerler: {[values[i] for i in selected_items]}")
print(f"⚖️ Toplam Ağırlık: {sum(weights[i] for i in selected_items)}")


✅ Optimal Değer: 1688692
📋 Seçilen Öğeler (0-index): [27, 40, 52, 87, 174, 203, 254, 255]
💰 Seçilen Değerler: [156840, 226844, 395124, 65488, 387480, 104036, 236380, 116500]
⚖️ Toplam Ağırlık: 4040184


In [6]:
save_to_excel("300.xlsx", 300, 1688692, selected_items, n)

✅ Excel dosyasına eklendi: 300.xlsx


In [3]:
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
import os

def save_to_excel(filename, size, total_value, selected_items, n_items):
    # 0/1 dizisini oluştur
    binary_solution = ['1' if i in selected_items else '0' for i in range(n_items)]
    binary_string = ",".join(binary_solution)
    selected_string = ",".join(str(i) for i in selected_items)

    # Dosya varsa aç, yoksa oluştur
    if os.path.exists(filename):
        wb = openpyxl.load_workbook(filename)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        # Başlık satırını yaz
        ws.append(["Dosya Boyutu", "Optimal Value Değeri", "Optimal çözüm", "Optimal çözüme dahil edilen itemler"])

    # Veri satırını ekle
    ws.append([size, total_value, binary_string, selected_string])
    
    # Sütun genişliklerini otomatik ayarla
    for col in ws.columns:
        max_len = max(len(str(cell.value)) for cell in col)
        ws.column_dimensions[get_column_letter(col[0].column)].width = max_len + 2

    wb.save(filename)
    print(f"✅ Excel dosyasına eklendi: {filename}")
